### Pre-process HMDB data

In [1]:
import argparse
import pandas as pd
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import sys
sys.path.append('..')

from molecules.utils import one_hot_array, one_hot_index

MAX_NUM_ROWS = 500000
SMILES_COL_NAME = 'structure'

The following are the command-line parameters of the **preprocess.py** script

In [3]:
# infile = '../data/smiles_50k.h5'
outfile = '../data/processed.h5'
smiles_column = SMILES_COL_NAME
length = MAX_NUM_ROWS

Load HMDB data

In [4]:
import numpy as np
import pylab as plt
import xmltodict
import traceback

In [5]:
class Metabolite(object):

    def __init__(self, name, formula, monoisotopic_mass, description, identifiers, taxonomy):
        self.name = name
        self.formula = formula
        self.monoisotopic_mass = monoisotopic_mass
        self.identifiers = identifiers
        self.taxonomy = taxonomy
        self.description = description
        self.class_id = -1

    def __str__(self):
        return '%s (%s)' % (self.name, self.formula)

In [ ]:
# data = xmltodict.parse(open('/Users/joewandy/Dropbox/Analysis/neural_net/hmdb/hmdb_metabolites.xml'))

In [ ]:
# compounds = []
# for node in data['hmdb']['metabolite']:
    
#     name = node['name']
#     formula = node['chemical_formula']
#     monoisotopic_mass = node['monisotopic_molecular_weight']

#     cas = node['cas_registry_number']
#     inchi = node['inchi']
#     inchikey = node['inchikey']
#     smiles = node['smiles']
#     identifiers = {
#         'cas': cas,
#         'inchi': inchi,
#         'inchikey': inchikey,
#         'smiles': smiles
#     }
    
#     tree = {}
#     description = None
#     if 'taxonomy' in node:
        
#         taxonomy = dict(node['taxonomy'])

#         description = taxonomy['description']        
#         direct_parent = taxonomy['direct_parent']
#         kingdom = taxonomy['kingdom']
#         super_class = taxonomy['super_class']
#         cclass = taxonomy['class']
#         sub_class = taxonomy['sub_class']
#         molecular_framework = taxonomy['molecular_framework']
#         tree = {
#             'kingdom': kingdom,
#             'super_class': super_class,
#             'class': cclass,
#             'sub_class': sub_class,
#             'molecular_framework': molecular_framework
#         }

#     if identifiers['smiles'] == None:
#         continue
#     else:
#         identifiers['smiles'] = identifiers['smiles'].encode('ascii', 'ignore')
        
#     compound = Metabolite(name, formula, monoisotopic_mass, description, identifiers, tree)
#     compounds.append(compound)

In [6]:
import cPickle

# with open('/Users/joewandy/Dropbox/Analysis/neural_net/hmdb_all_compounds.p','wb') as f:
#     cPickle.dump(compounds, f)
    
with open('/Users/joewandy/Dropbox/Analysis/neural_net/hmdb_all_compounds.p','rb') as f:
    compounds = cPickle.load(f)

In [15]:
print len(compounds)
for i in range(100):
    print compounds[i], compounds[i].identifiers['smiles']

46124
1-Methylhistidine (C7H11N3O2) CN1C=NC(C[C@H](N)C(O)=O)=C1
1,3-Diaminopropane (C3H10N2) NCCCN
2-Ketobutyric acid (C4H6O3) CCC(=O)C(O)=O
2-Hydroxybutyric acid (C4H8O3) CCC(O)C(O)=O
2-Methoxyestrone (C19H24O3) [H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[C@@]21[H])C=C(O)C(OC)=C3
(R)-3-Hydroxybutyric acid (C4H8O3) C[C@@H](O)CC(O)=O
Deoxyuridine (C9H12N2O5) OC[C@H]1O[C@H](C[C@@H]1O)N1C=CC(=O)NC1=O
Deoxycytidine (C9H13N3O4) NC1=NC(=O)N(C=C1)[C@H]1C[C@H](O)[C@@H](CO)O1
Cortexolone (C21H30O4) [H]C12CC[C@](O)(C(=O)CO)[C@@]1(C)CCC1([H])[C@@]2([H])CCC2=CC(=O)CC[C@]12C
Deoxycorticosterone (C21H30O3) [H][C@@]12CC[C@H](C(=O)CO)[C@@]1(C)CC[C@@]1([H])[C@@]2([H])CCC2=CC(=O)CC[C@]12C
4-Pyridoxic acid (C8H9NO4) CC1=NC=C(CO)C(C(O)=O)=C1O
Alpha-ketoisovaleric acid (C5H8O3) CC(C)C(=O)C(O)=O
p-Hydroxyphenylacetic acid (C8H8O3) OC(=O)CC1=CC=C(O)C=C1
Iodotyrosine (C9H10INO3) N[C@@H](CC1=CC=C(O)C(I)=C1)C(O)=O
3-Methoxytyramine (C9H13NO2) COC1=C(O)C=CC(CCN)=C1
(S)-3-Hydroxyisobutyric acid (C4H8O3) C[C@@H]

In [19]:
last = compounds[-100:]
for c in last:
    print c, c.identifiers['smiles']

TG(20:0/20:0/21:0) (C64H124O6) [H][C@](COC(=O)CCCCCCCCCCCCCCCCCCC)(COC(=O)CCCCCCCCCCCCCCCCCCCC)OC(=O)CCCCCCCCCCCCCCCCCCC
TG(a-21:0/14:0/i-17:0)[rac] (C55H106O6) [H][C@](COC(=O)CCCCCCCCCCCCCCCCC(C)CC)(COC(=O)CCCCCCCCCCCCCC(C)C)OC(=O)CCCCCCCCCCCCC
TG(a-21:0/a-13:0/22:0)[rac] (C59H114O6) [H][C@@](COC(=O)CCCCCCCCCCCCCCCCCCCCC)(COC(=O)CCCCCCCCCCCCCCCCC(C)CC)OC(=O)CCCCCCCCC(C)CC
TG(a-21:0/i-14:0/14:0)[rac] (C52H100O6) [H][C@@](COC(=O)CCCCCCCCCCCCC)(COC(=O)CCCCCCCCCCCCCCCCC(C)CC)OC(=O)CCCCCCCCCCC(C)C
TG(a-21:0/17:0/14:0)[rac] (C55H106O6) [H][C@@](COC(=O)CCCCCCCCCCCCC)(COC(=O)CCCCCCCCCCCCCCCCC(C)CC)OC(=O)CCCCCCCCCCCCCCCC
TG(a-21:0/i-17:0/13:0)[rac] (C54H104O6) [H][C@@](COC(=O)CCCCCCCCCCCC)(COC(=O)CCCCCCCCCCCCCCCCC(C)CC)OC(=O)CCCCCCCCCCCCCC(C)C
TG(21:0/21:0/16:0) (C61H118O6) [H][C@@](COC(=O)CCCCCCCCCCCCCCC)(COC(=O)CCCCCCCCCCCCCCCCCCCC)OC(=O)CCCCCCCCCCCCCCCCCCCC
TG(20:0/i-24:0/i-24:0) (C71H138O6) [H][C@](COC(=O)CCCCCCCCCCCCCCCCCCC)(COC(=O)CCCCCCCCCCCCCCCCCCCCC(C)C)OC(=O)CCCCCCCCCCCCCCCCCCCCC(C)C

Turn into a panda thing

In [8]:
rows = [compound.identifiers['smiles'][:120] for compound in compounds]
df = pd.DataFrame(rows, columns=['structure'])

In [9]:
df

,structure
0,CN1C=NC(C[C@H](N)C(O)=O)=C1
1,NCCCN
2,CCC(=O)C(O)=O
3,CCC(O)C(O)=O
4,[H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[...
5,C[C@@H](O)CC(O)=O
6,OC[C@H]1O[C@H](C[C@@H]1O)N1C=CC(=O)NC1=O
7,NC1=NC(=O)N(C=C1)[C@H]1C[C@H](O)[C@@H](CO)O1
8,[H]C12CC[C@](O)(C(=O)CO)[C@@]1(C)CCC1([H])[C@@...
9,[H][C@@]12CC[C@H](C(=O)CO)[C@@]1(C)CC[C@@]1([H...


Find rows containing SMILES < 121 chars

In [10]:
keys = df[smiles_column].map(len) < 121
print len(keys)
print np.sum(keys.values)

46124
46124


If too many rows (>MAX_NUM_ROWS), then sample up to MAX_NUM_ROWS rows)

In [11]:
if length <= len(keys):
    df = df[keys].sample(n = args.length)
else:
    df = df[keys]

Tokenize each SMILE string. Pad to 120 chars.

In [12]:
structures = df[smiles_column].map(lambda x: list(x.ljust(120)))

In [13]:
structures

0        [C, N, 1, C, =, N, C, (, C, [, C, @, H, ], (, ...
1        [N, C, C, C, N,  ,  ,  ,  ,  ,  ,  ,  ,  ,  , ...
2        [C, C, C, (, =, O, ), C, (, O, ), =, O,  ,  , ...
3        [C, C, C, (, O, ), C, (, O, ), =, O,  ,  ,  , ...
4        [[, H, ], [, C, @, @, ], 1, 2, C, C, C, (, =, ...
5        [C, [, C, @, @, H, ], (, O, ), C, C, (, O, ), ...
6        [O, C, [, C, @, H, ], 1, O, [, C, @, H, ], (, ...
7        [N, C, 1, =, N, C, (, =, O, ), N, (, C, =, C, ...
8        [[, H, ], C, 1, 2, C, C, [, C, @, ], (, O, ), ...
9        [[, H, ], [, C, @, @, ], 1, 2, C, C, [, C, @, ...
10       [C, C, 1, =, N, C, =, C, (, C, O, ), C, (, C, ...
11       [C, C, (, C, ), C, (, =, O, ), C, (, O, ), =, ...
12       [O, C, (, =, O, ), C, C, 1, =, C, C, =, C, (, ...
13       [N, [, C, @, @, H, ], (, C, C, 1, =, C, C, =, ...
14       [C, O, C, 1, =, C, (, O, ), C, =, C, C, (, C, ...
15       [C, [, C, @, @, H, ], (, C, O, ), C, (, O, ), ...
16       [C, C, C, C, C, C, C, C, C, C, C, C, C, C, C, .

In [14]:
np.array(structures[0])

array(['C', 'N', '1', 'C', '=', 'N', 'C', '(', 'C', '[', 'C', '@', 'H',
       ']', '(', 'N', ')', 'C', '(', 'O', ')', '=', 'O', ')', '=', 'C',
       '1', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' '], 
      dtype='|S1')

Split into training-testing sets

In [15]:
train_idx, test_idx = map(np.array, train_test_split(structures.index, test_size = 0.20))

In [16]:
print len(train_idx)
print len(test_idx)

36899
9225


Find the unique tokens

In [17]:
charset = list(reduce(lambda x, y: set(y) | x, structures, set()))
print charset

[' ', '(', '.', '0', '2', '4', '6', '8', '@', 'B', 'F', 'H', 'L', 'N', 'P', 'R', 'T', 'V', 'Z', '\\', 'b', 'd', 'f', 'h', 'l', 'n', 'r', 't', '#', '%', ')', '+', '-', '/', '1', '3', '5', '7', '9', '=', 'A', 'C', 'G', 'I', 'K', 'M', 'O', 'S', 'W', 'Y', '[', ']', 'a', 'e', 'g', 'i', 'm', 'o', 's', 'u']


Turn into one-hot encoded vectors. Save the output into another hdf5 file.

In [18]:
def chunk_iterator(dataset, chunk_size=1000):
    chunk_indices = np.array_split(np.arange(len(dataset)),
                                    len(dataset)/chunk_size)
    for chunk_ixs in chunk_indices:
        chunk = dataset[chunk_ixs]
        yield (chunk_ixs, chunk)
    raise StopIteration

In [19]:
def create_chunk_dataset(h5file, dataset_name, dataset, dataset_shape,
                         chunk_size=1000, apply_fn=None):
    new_data = h5file.create_dataset(dataset_name, dataset_shape,
                                     chunks=tuple([chunk_size]+list(dataset_shape[1:])))
    for (chunk_ixs, chunk) in chunk_iterator(dataset):
        if not apply_fn:
            new_data[chunk_ixs, ...] = chunk
        else:
            new_data[chunk_ixs, ...] = apply_fn(chunk)

In [20]:
one_hot_encoded_fn = lambda row: map(lambda x: one_hot_array(x, len(charset)),
                                            one_hot_index(row, charset))

In [21]:
h5f = h5py.File(outfile, 'w')
h5f.create_dataset('charset', data = charset)

create_chunk_dataset(h5f, 'data_train', train_idx,
                     (len(train_idx), 120, len(charset)),
                     apply_fn=lambda ch: np.array(map(one_hot_encoded_fn,
                                                      structures[ch])))
create_chunk_dataset(h5f, 'data_test', test_idx,
                     (len(test_idx), 120, len(charset)),
                     apply_fn=lambda ch: np.array(map(one_hot_encoded_fn,
                                                      structures[ch])))

In [22]:
h5f.close()